In [1]:
import os

In [2]:
%pwd

'/home/joshua/Customer-Segmentation-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/joshua/Customer-Segmentation-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/JoshuaVictorL/Customer-Segmentation-Project.mlflowmlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="JoshuaVictorL"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ba3573d2f61c38b6a9def21419a4bc92446acfbb"

In [10]:
import os
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: str
    model_path: str
    test_data_path: str
    evaluation_metrics_path: str
    mlflow_uri: str
    target_column: str
    all_params: dict

In [11]:
from customer_segmentation.constants import *
from customer_segmentation.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create main directory
        create_directories([self.config['model_evaluation']['root_dir']])

    def get_model_evaluation_config(self):
        config = self.config['model_evaluation']
        params = self.params['model_trainer']

        create_directories([config['root_dir']])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config['root_dir'],
            model_path=config['model_path'],
            test_data_path=config['test_data_path'],
            evaluation_metrics_path=config['evaluation_metrics_path'],
            mlflow_uri=config['mlflow_uri'],
            target_column=config['target_column'],
            all_params=params
        )

        return model_evaluation_config


In [12]:
# Model Evaluation Pipeline with MLflow Integration

import pandas as pd
import joblib
import mlflow
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pathlib import Path
import json
import yaml

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average='weighted')
        recall = recall_score(actual, pred, average='weighted')
        f1 = f1_score(actual, pred, average='weighted')
        return accuracy, precision, recall, f1

    def save_json(self, path, data):
        with open(path, 'w') as f:
            json.dump(data, f, indent=4)

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_clusters = model.predict(test_x)
            
            accuracy, precision, recall, f1 = self.eval_metrics(test_y, predicted_clusters)

            # Save metrics locally
            scores = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1_score": f1
            }
            self.save_json(Path(self.config.evaluation_metrics_path), scores)

            # Log parameters from params.yaml and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)

            # Model registry (if not file store)
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestSegmentModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-23 18:54:20,701: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-23 18:54:20,709: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 18:54:20,712: INFO: common: created directory at: artifacts/model_evaluation]
[2025-02-23 18:54:20,715: INFO: common: created directory at: artifacts/model_evaluation]


[2025-02-23 18:56:20,940: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /JoshuaVictorL/Customer-Segmentation-Project.mlflow/api/2.0/mlflow/runs/create]


/home/joshua/.virtualenvs/ml/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'RandomForestSegmentModel'.
2025/02/23 18:56:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestSegmentModel, version 1
Created version '1' of model 'RandomForestSegmentModel'.
